<a href="https://colab.research.google.com/github/ufrpe-ensino/curso-mineracao-textos/blob/master/05_Gensim_W2V_Embeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Treinando uma representação de Embeddings de texto com Gensim

In [ ]:
from gensim.models import Word2Vec
import multiprocessing

**Corpus**

In [ ]:
sentences = [
      ['van', 'gogh', 'nasceu', 'numa', 'família', 'de', 'classe', 'média', 'alta'],
      ['começou', 'a', 'desenhar', 'ainda', 'criança', 'sendo', 'descrito', 'como', 'alguém', 'sério', ',', 'quieto', 'e', 'pensativo'],
      ['vincent', 'willem', 'van', 'gogh', 'gostava', 'de', 'desenhar', 'em', 'zundert'],
      ['vincent', 'era', 'um', 'nome', 'comum', 'na', 'família', 'van', 'gogh'],
      ['era', 'o', 'filho', 'mais', 'velho', 'sobrevivente', 'de', 'anna', 'cornelia', 'carbentus', 'e', 'theodorus', '.']
      
]

**Treinando Modelo**

In [ ]:
model = Word2Vec(sentences,
                 sg=1, #0 para CBOW
                 size=300,
                 workers=multiprocessing.cpu_count(),
                 iter=5,
                 negative=5,
                 min_count=1)

**Vetor da palavra "vincent"**

In [ ]:
model["vincent"][0:5]

**Armazenando todos vetores do Modelo**

In [ ]:
x = model[model.wv.vocab]

**Redução da Dimensionalidade**

In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components=2)

dim = pca.fit_transform(x)

**Plot do Modelo**

In [ ]:
from matplotlib import pyplot as plt

plt.figure(figsize=(15,15))
plt.scatter(dim[:, 0], dim[:, 1])

words = list(model.wv.vocab)
for i, word in enumerate(words):
	plt.annotate(word, xy=(dim[i, 0], dim[i, 1]))
plt.show()

**Salvando os embeddings gerados**

In [ ]:
model.save('model.bin')

model.wv.save_word2vec_format("model_skpg_300d.txt")

**Carregando Modelo .bin**

In [ ]:
model_bin = Word2Vec.load('model.bin')

model_bin['gogh'][0:5]

**Carregando Modelo .txt**

In [ ]:
from gensim.models import KeyedVectors

model_txt = KeyedVectors.load_word2vec_format('model_skpg_300d.txt')

model_txt['gogh'][0:5]

In [ ]:
model_txt.most_similar('vincent')

## Exercício
Gere uma visualização dos embeddings produzidos pelo corpus do livro de Memórias Póstumas de Brás Cubas, no corpus do NLTK

In [ ]:
import nltk
nltk.download('machado')
nltk.download('punkt')

from nltk.corpus import machado
print(machado.readme())

In [ ]:
import numpy as np
import os
from nltk.corpus import machado
import unicodedata

# Remove acentos e coloca palavras em minúsculas
def strip_accents_and_lower(s):
    return ''.join(c for c in unicodedata.normalize('NFD', s) if unicodedata.category(c) != 'Mn').lower()

machado_sents = map(lambda sent: list(map(strip_accents_and_lower, sent)), machado.sents())

# 'Executa' o mapeamento da lista
%time machado_sents = list(machado_sents)

Utilize as sentenças da variável `machado_sents` para treinar e visualizar o embedding produzido

In [ ]:
# sua resposta

Exiba as palavras mais próximas da lista abaixo

In [ ]:
# Exibe algumas palavras próximas daquelas contidas nesta lista
test_words = ['seja', 'foi', 'amou', 'aquele', 'foram', 'homem', 'rua', 'marcela']

# Embedding de uma palavra
def word_embedding(word):
    return model.wv[word]

# Pega apenas as palavras a partir do resultado da função 'most_similar'
def strip_score(result):
    return [w for w, s in result]
    
# Lista as palavras mais próximas
def closest_words(word, num=5):
    word_score_pair = model.wv.most_similar(word, topn=num)
    return strip_score(word_score_pair)

for w in test_words:
    print(w, closest_words(w))